In [1]:
%matplotlib inline 
import matplotlib.pylab as plt 
import numpy as np 
import pandas as pd

from datetime import date
from datetime import datetime 
# import h5py
import os 


# --- these two packages are used to work with the GIS (Geographic Information System) data --- 
import fiona
import rasterio
import rasterio.mask 
import rasterstats   # not used in first attempt actually 

#  ---- my functions ---- 
import GIS_utils  

from importlib import reload
reload(GIS_utils);

# my notes 

'fiona' package can open the shapefiles (.shp). 
Every entry into the shapefile has the following properties I would want to use: 
* geometry: the main shape data to use for overlapping against the tiff file 
    contains:
    1. type: 'Polygon' / 'line' etc. 
    2. 'coordinates' : THE points used to define the shape 
* REGIONNAME: admin lvl 1 
* ZONENAME: admin lvl 2 
* WOREDNAME: admin lvl 3 
* HRname: 


## plan: 
1. Open shape file using 'fiona' package 
    * extract the shapes / polygon outline coordinate data (admin boundaries)
    * extract admin names 
    
2. Open satelite image using 'rasterio' package 
    * for every polygon in the admin boundaries file (shapefile)
         * perform mask 
         * 'exclude the pixels outside of boundary' 
         * calculate disired metric ('min', 'max', 'median', whatever...)
         
3. Store everything into dataframe: 
    * district name 
    * date
    * value of metric
    
4. Finally create a HDF5:
    * Every dataset corresponds to 1 rasterfile / time point 
    * Every dataset therefore contains one dataframe 
    


# Dry matter productivity

Found this on the DMP source website. 

* units: kg/ha/day 
* physical min: 0 
* physical max: 327.67
* digital max: 32767
* scaling: 1/100 

And the following values are used as indicators: 
* -1	Background 
* -2	Sea 
* -3	Snow/Ice
* -4	Cloud
* -5	Missing data
* -257	NDVI < 0
* -300	Missing Meteo data


# util function for DMP data 

In [2]:
def decode_date_from_folder(name):
    '''
    DMV file folder names contained the date. 
    This function interprets these names and gets the name from them. 
    
    
    returns a string that can be converted by Pandas into a timestep of datetime 
    '''
    
    
    try:
        int(name.split('_')[2])
        encoded_date = name.split('_')[2]
    except:
        try: 
            int(name.split('_')[1])
            encoded_date = name.split('_')[1]
        except:
            encoded_date = name
    
    year = int(encoded_date[:4])
    month = int(encoded_date[4:6])
    day  = int(encoded_date[6:8])
    return datetime.strftime( date(year, month, day), format='%Y-%m-%d' )

# extract DMP data into HDF5 file 

In [19]:
# --- input data ---- 
path_to_GitRepo = '/Users/mklein1/Documents/RedCross510/Drought_IBF/'
path_to_HD = '/Volumes/BigData/RK_IBF_drought/DMP_Ethiopia/'
file_shp = path_to_GitRepo + 'shapefiles/Ethiopia/Administrative boundaries/ETH_adm1_mapshaper_reproj.shp'

# --- output data into HDF5 file --- 
file_out = path_to_HD + 'Ethiopia_DMP_adm1.hdf5'


# --- loop through the whole set of DMP data files/folders ---- 
folders = [name for name in os.listdir(path_to_HD) if os.path.isdir(path_to_HD+name) ]

for i, directory in enumerate(folders):
    files = [name for name in os.listdir(path_to_HD+directory) if name.endswith('.tiff')]
    file_raster = path_to_HD + directory + '/' + files[0]
    
    # --- should see what happens when I use the .nc files in stead of the .tiff ---- 
    if len(files) != 1:
        print(directory, len(files))
    
    
    
    # --- extract the data/ summerize per polygon into metrics --- 
    ZoneStats , _ , _ = GIS_utils.extract_zonal_stats(file_shp, file_raster,
                                                        minval= 0,
                                                        name_property='REGIONNAME',
                                                        pcode_property='REG_P_CODE')
    
    # ---- store output into HDF5 file --- 
    timestamp = decode_date_from_folder(directory)

    ZoneStats.to_hdf(file_out, key=timestamp, table=True, mode='a')
    
    
    print('produced dataset ', i+1, ' out of ', len(folders), end='\r')
#     if i == 3:
#         break
    
        
   

/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2010-01-10'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2010-01-20'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2010-01-31'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2010-02-10'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2010-02-20'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2010-02-28'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2010-03-10'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2010-03-20'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2010-03-31'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


...

/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2018-10-10'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2018-10-20'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


produced dataset 

/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2018-10-31'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2018-11-10'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2018-11-20'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/mklein1/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2018-11-30'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


IndexError: list index out of range

In [20]:
df = pd.read_hdf(file_out, key='2018-01-31')
df

,pcode,area,mean,min,max,med
0,14,Addis Ababa,10.093750,0,46,9.0
1,2,Afar,3.278314,0,79,0.0
2,3,Amhara,11.242857,0,90,0.0
3,6,Beneshangul Gumu,7.776471,0,76,6.0
4,15,Dire Dawa,8.416667,0,48,0.0
5,12,Gambela,6.325043,0,105,0.0
6,13,Hareri,16.333333,0,50,21.0
7,4,Oromia,12.771874,0,144,0.0
8,7,SNNPR,16.544322,0,105,3.0
9,5,Somali,2.759618,0,48,0.0


In [2]:
import h5py 
import netCDF4
path_to_HD = '/Volumes/BigData/RK_IBF_drought/DMP_Ethiopia/'
directory = 'DMP_RT6_201812100000_GLOBE_PROBAV_V2.0.1'

files = [name for name in os.listdir(path_to_HD+directory)]
print(files)


src = netCDF4.Dataset(path_to_HD+directory+'/'+files[0],'r')

# with rasterio.open('netcdf:'+path_to_HD+directory+'/'+files[0], 'r') as src:

#     # --- acces the image (the matrix with intensity values) ---- 
#     img = src.read(1)

# #     # --- show the image ---- 
# #     plt.imshow(img)
    


['c_gls_DMP-RT6_201812100000_GLOBE_PROBAV_V2.0.1.nc']


OSError: [Errno -101] NetCDF: HDF error: b'/Volumes/BigData/RK_IBF_drought/DMP_Ethiopia/DMP_RT6_201812100000_GLOBE_PROBAV_V2.0.1/c_gls_DMP-RT6_201812100000_GLOBE_PROBAV_V2.0.1.nc'

In [ ]:
path_to_HD+directory+'/'+files[0]

In [ ]:

# with h5py.File(destination+fname, 'w') as f:
    
#     RasterData.to_hdf(destination+fname, timestamp, table=True, mode='a')
    
    
df

In [ ]:
decode_date_from_folder(fldr_name_2)

In [ ]:

import os 



In [ ]:
path = "/Users/mklein1/Desktop/"
file_shp = path + 'Administrative boundaries/ETH_adm2_mapshaper_reproj.shp'
file_tiff= path + 'c_gls_DMP-DMP_201001100000_CUSTOM_VGT_V2.0.1.tiff'

z_stats = rasterstats.zonal_stats(file_shp,file_tiff,stats=['min', 'max', 'mean', 'median'])

df = pd.DataFrame(z_stats)
df

In [ ]:
reload(GIS_utils)
path = "/Users/mklein1/Desktop/"
file_shp = path + 'Administrative boundaries/ETH_adm2_mapshaper_reproj.shp'
file_tiff= path + 'c_gls_DMP-DMP_201001100000_CUSTOM_VGT_V2.0.1.tiff'

RasterData , data , img = GIS_utils.extract_data_raster(file_shp, file_tiff,
                                                        minval= 0,
#                                                         maxval= 32767,
                                                        name_property='ZONENAME',
                                                        pcode_property='ZON_P_CODE')
RasterData

In [ ]:
with rasterio.open(file_tiff, 'r') as src:
        # --- acces the image (the matrix with intensity values) ----
        print(src.nodata)
        
        img = src.read(1)
plt.imshow(img,cmap='hot') 
plt.colorbar()

In [ ]:
df = pd.DataFrame()
df['A'] = avg_vals
df['B'] = min_vals
df['C'] = max_vals
df['D'] = names
df.head()

In [ ]:
sum( df['D'] == 140107)

In [ ]:
with fiona.open("/Users/mklein1/Desktop/Administrative boundaries/ETH_adm2_mapshaper_reproj.shp", "r") as shapefile:
    for feature in shapefile:
#         print(feature['properties']['WOREDANAME'])
        print(feature)
        print('\n')
        break
#     shapes = [feature["geometry"] for feature in shapefile]
   

In [ ]:
path = "/Users/mklein1/Desktop/"
file_shp = 'Administrative boundaries/ETH_adm3_mapshaper_reproj.shp'
file_tiff= 'c_gls_DMP-DMP_201001100000_CUSTOM_VGT_V2.0.1.tiff'





    
#     out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
#     out_meta = src.meta

In [ ]:
# --- use rasterio to read the satelite image (raster data/vector image) ---- 
with rasterio.open(path+file_tiff,'r') as src:

    # --- acces the image (the matrix with intensity values) ---- 
    img = src.read(1)

    # --- show the image ---- 
    plt.imshow(img)
    
    # --- perform the mask operation for every shape in the shapefile --- 
    
    for shape in shapes: 
        out_image, out_transform = rasterio.mask.mask(src, [shape], crop=True)
        
        # --- show the masked image (shows non-zero value within boundaries, zero outside of boundaries shape) ---- 
        img = out_image[0,:,:]
        # --- filter out those pixels that fall outside the boundaries of the clipping mask 
        # ---- (these are set to negative values) ----
        data = img[img>img.min()]
        print('mean: ', data.mean())
        print('min: ', data.min())
        print('max: ', data.max())
        print('\n')
        
        
#         plt.figure()
#         plt.imshow(out_image[0,:,:])

In [ ]:
adm_lvl3

In [ ]:
for shape in shapes:
    print(shape)
    break 